In [1]:
# Core libs
import pandas as pd
import numpy as np
import importlib

# Graphic libs
import plotly.express as px
import plotly.io as pio
import matplotlib.pyplot as plt
import seaborn as sns
pio.renderers.default = "browser"

# sklearn 
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.metrics import mean_absolute_percentage_error as LMAPE

# IA + blockchain = profit
import tensorflow as tf
from keras import optimizers
from keras.models import Model
from keras.layers import Input, Lambda, Dropout, BatchNormalization, Activation, Dense, LeakyReLU, Add
from keras.layers.merging import Add, Concatenate
from keras.utils import plot_model, to_categorical

2022-12-08 21:17:31.384474: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-08 21:17:31.518322: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-08 21:17:32.172726: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-08 21:17:32.172787: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
raw_data_master = pd.read_excel('../data/xlsx/ord80p.xlsx', header = [0]).applymap(lambda x : (x - pd.Timestamp("1900-01-01 00:00:00")).days + 2 if isinstance(x,pd.Timestamp) else x).fillna(-1)
raw_column_names = raw_data_master.columns

In [6]:
raw_data_master['Categoría del bien'].count()

42604

In [3]:
ordinal_data = raw_data_master[raw_data_master['Provincia'] == 'Lima'].copy()

for column in raw_column_names[:-2]:
    dist_encod = OrdinalEncoder()

    if column in ['Fecha entrega del Informe', 'Tipo de vía','Latitud (Decimal)', 'Longitud (Decimal)', 'Número de frentes', 'Edad', 'Elevador','Área Terreno','Área Construcción', 'Valor comercial (USD)', 'ranva']:
        continue

    enc = dist_encod.fit_transform(ordinal_data[[column,'ranva']].to_numpy(dtype = str))
    ordinal_data[column] = enc.T[0]

ordinal_data = ordinal_data.applymap(lambda x : float(x.replace(',','')) if isinstance(x,str) else x)
ordinal_data = ordinal_data.applymap(lambda x : np.cbrt(x))


In [4]:
SIMPLE_COLUMNS = [
    'Fecha',
    'TipoDeVia',
    'Piso',
    'Departamento',
    'Provincia',
    'Distrito',
    'Estacionamiento',
    'Depositos',
    'Lat',
    'Lon',
    'Categoria',
    'Posicion',
    'Frentes',
    'Edad',
    'Elevador',
    'Conservacion',
    'Metodo',
    'AreaTerreno',
    'AreaConstruccion',
    'Valorcomercial',
    'ranv'
]

In [110]:
ordinal_data.columns = SIMPLE_COLUMNS

In [111]:
ordinal_data_nv = ordinal_data.drop(['Valorcomercial', 'ranv'], axis=1)

In [112]:
ordinal_data

,Fecha,TipoDeVia,Piso,Departamento,Provincia,Distrito,Estacionamiento,Depositos,Lat,Lon,...,Posicion,Frentes,Edad,Elevador,Conservacion,Metodo,AreaTerreno,AreaConstruccion,Valorcomercial,ranv
1,35.107695,1.442250,0.0,0.0,0.0,3.419952,1.000000,1.000000,-2.297598,-4.253887,...,0.0,-1.0,1.709976,-1.0,1.000000,1.000000,2.334331,4.544739,54.337732,-1.0
2,35.194294,1.259921,0.0,0.0,0.0,3.207534,1.000000,1.000000,-2.299705,-4.253772,...,0.0,-1.0,2.519842,-1.0,1.912931,1.259921,5.428835,7.642605,66.536716,-1.0
3,35.178409,1.442250,0.0,0.0,0.0,3.448217,1.259921,1.000000,-2.296555,-4.254586,...,0.0,-1.0,1.000000,-1.0,1.442250,1.000000,1.579424,4.406464,49.966511,-1.0
5,35.049997,1.442250,0.0,0.0,0.0,3.301927,1.000000,1.000000,-2.293414,-4.256021,...,0.0,-1.0,1.259921,-1.0,1.000000,1.000000,1.775808,4.180484,45.973677,-1.0
7,35.127697,1.000000,0.0,0.0,0.0,3.141381,1.259921,1.000000,-2.296193,-4.254534,...,0.0,-1.0,3.107233,-1.0,1.000000,1.000000,1.531198,2.656646,23.012595,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42597,34.915169,1.259921,0.0,0.0,0.0,3.271066,1.000000,1.259921,-2.291514,-4.255822,...,0.0,-1.0,1.709976,-1.0,0.000000,1.000000,3.562680,4.284313,41.517146,-1.0
42598,35.067624,1.259921,0.0,0.0,0.0,2.519842,1.000000,1.000000,-2.294748,-4.255135,...,0.0,-1.0,3.448217,-1.0,1.000000,1.000000,3.367863,4.530655,48.805000,-1.0
42601,35.384868,1.442250,0.0,0.0,0.0,2.351335,2.289428,1.000000,-2.294646,-4.253534,...,0.0,-1.0,2.289428,-1.0,1.000000,1.000000,4.947858,5.707756,64.449473,-1.0
42602,34.902860,1.000000,0.0,0.0,0.0,3.476027,1.000000,1.000000,-2.303601,-4.253137,...,0.0,-1.0,2.519842,-1.0,0.000000,1.000000,5.192494,6.542133,45.705746,-1.0


In [113]:
#ordinal_data.upadte('Valorcomercial') = np.cbrt(ordinal_data['Valorcomercial'].to_numpy())
ordinal_data_nv.describe()

,Fecha,TipoDeVia,Piso,Departamento,Provincia,Distrito,Estacionamiento,Depositos,Lat,Lon,Categoria,Posicion,Frentes,Edad,Elevador,Conservacion,Metodo,AreaTerreno,AreaConstruccion
count,27426.000000,27426.000000,27426.000000,27426.0,27426.0,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000,27426.000000
mean,35.164087,1.107249,0.001015,0.0,0.0,2.716177,1.008289,0.901841,-2.276918,-4.208255,1.751689,0.002729,-0.995703,1.853633,-0.994792,0.977539,1.066549,4.108764,5.054914
std,0.150495,0.539034,0.042583,0.0,0.0,0.642641,0.549679,0.423334,0.151824,0.387456,0.525521,0.053418,0.093545,1.317892,0.102637,0.637262,0.113989,3.751524,2.479996
min,34.780945,-1.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,-4.255516,-9.129394,0.000000,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,-1.000000,-1.000000
25%,35.057863,1.000000,0.000000,0.0,0.0,2.410142,1.000000,1.000000,-2.296739,-4.255283,1.587401,0.000000,-1.000000,1.000000,-1.000000,1.000000,1.000000,2.251275,4.181294
50%,35.169518,1.259921,0.000000,0.0,0.0,2.758924,1.000000,1.000000,-2.295037,-4.254722,1.587401,0.000000,-1.000000,1.912931,-1.000000,1.000000,1.000000,3.050264,4.747459
75%,35.267877,1.442250,0.000000,0.0,0.0,3.271066,1.259921,1.000000,-2.293450,-4.254047,2.154435,0.000000,-1.000000,2.843867,-1.000000,1.442250,1.259921,5.267740,5.862571
max,35.534384,2.154435,2.351335,0.0,0.0,3.503398,3.391211,2.843867,0.000000,0.000000,2.466212,1.259921,2.802039,12.609701,1.259921,2.000000,1.442250,215.443469,35.523294


In [57]:
out = ['Departamento','Piso','Posicion','Provincia','Frentes','Elevador','ranv']
plot_vars = []
for var in SIMPLE_COLUMNS:
    if var not in out:
        plot_vars.append(var)

fig = px.scatter_matrix(ordinal_data.drop(out, axis=1),
                        dimensions=plot_vars,
                        title="Scatter matrix of data set",
                        color='Valorcomercial',
                        labels=plot_vars) # remove underscore
fig.update_traces(diagonal_visible=False)
fig.update_traces(marker_size=1)

fig.update_layout(font=dict(size=10))

fig.update_layout({"xaxis"+str(i+1): dict(showticklabels = False, title=plot_vars[i][:3].upper()) for i in range(len(plot_vars))})
fig.update_layout({"yaxis"+str(i+1): dict(showticklabels = False, title=plot_vars[i][:3].upper()) for i in range(len(plot_vars))})

fig.update_layout(margin={"r":150,"t":20,"l":150,"b":20})
fig.show()

In [13]:
ordinal_data

,Fecha,TipoDeVia,Piso,Departamento,Provincia,Distrito,Estacionamiento,Depositos,Lat,Lon,...,Posicion,Frentes,Edad,Elevador,Conservacion,Metodo,AreaTerreno,AreaConstruccion,Valorcomercial,ranv
0,35.270824,1.442250,0.0,2.410142,4.973190,7.422914,-1.000000,-1.0,-2.326799,-4.105125,...,0.0,-1.0,2.466212,-1.0,1.000000,1.259921,24.216218,18.437824,55.353862,-1.0
1,35.107695,1.442250,0.0,2.289428,4.290840,7.281079,0.000000,0.0,-2.297598,-4.253887,...,0.0,-1.0,1.709976,-1.0,1.000000,1.000000,12.711598,26.411118,54.337732,-1.0
2,35.194294,1.259921,0.0,2.289428,4.290840,7.113787,0.000000,0.0,-2.299705,-4.253772,...,0.0,-1.0,2.519842,-1.0,1.912931,1.259921,15.378516,23.301021,66.536716,-1.0
3,35.178409,1.442250,0.0,2.289428,4.290840,7.386437,1.000000,0.0,-2.296555,-4.254586,...,0.0,-1.0,1.000000,-1.0,1.442250,1.000000,20.139031,25.980261,49.966511,-1.0
4,35.360359,1.000000,0.0,2.223980,4.217163,5.924992,0.000000,0.0,-1.885117,-4.307105,...,0.0,-1.0,0.000000,-1.0,1.587401,1.259921,23.520580,20.880653,28.643678,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42599,35.131748,1.442250,0.0,1.709976,4.672329,3.684031,0.000000,0.0,-1.000000,-1.000000,...,0.0,-1.0,-1.000000,-1.0,0.000000,1.259921,0.000000,0.000000,22.385093,-1.0
42600,35.228706,1.000000,0.0,2.571282,4.657010,7.559526,0.000000,0.0,-1.728380,-4.321086,...,0.0,-1.0,0.000000,-1.0,1.442250,1.259921,23.520580,20.880653,29.338483,-1.0
42601,35.384868,1.442250,0.0,2.289428,4.290840,5.848035,1.259921,0.0,-2.294646,-4.253534,...,0.0,-1.0,2.289428,-1.0,1.000000,1.000000,12.850355,18.312446,64.449473,-1.0
42602,34.902860,1.000000,0.0,2.289428,4.290840,7.582787,0.000000,0.0,-2.303601,-4.253137,...,0.0,-1.0,2.519842,-1.0,0.000000,1.000000,14.204482,21.076066,45.705746,-1.0


In [116]:
data_pca = PCA()
data_pca.fit(ordinal_data[raw_data_master['Provincia'] == 'Lima'].drop(out, axis=1))
X = (ordinal_data[raw_data_master['Provincia'] == 'Lima'].drop(out, axis=1).to_numpy()).T
components = (data_pca.components_@X).T

components.shape

/tmp/ipykernel_29960/2778599912.py:2: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/tmp/ipykernel_29960/2778599912.py:3: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



(27426, 14)

In [117]:
components_df = pd.DataFrame(components, columns=['PC'+str(i+1)+' ({:.1f})%'.format(data_pca.explained_variance_ratio_[i]*100) for i in range(components.shape[1])])
#components_df.insert(0, 'Valorcomercial', trainer_data['Valorcomercial'].to_numpy())
#components_df.insert(0, 'Valorcomercial', ordinal_data[raw_data_master['Provincia'] == 'Lima']['Valorcomercial'])

fig = px.scatter_matrix(components_df,
    dimensions=['PC'+str(i+1)+' ({:.1f})%'.format(data_pca.explained_variance_ratio_[i]*100) for i in range(components.shape[1])],
    color=ordinal_data['Categoria'],#np.array([val if val < 200 else 300 for val in mape_list]),
    title="PCA Pairs plot",
    labels=components_df.columns) # remove underscore
fig.update_traces(diagonal_visible=False)
fig.update_traces(marker_size=1)

fig.update_layout(font=dict(size=10))

fig.update_layout({"xaxis"+str(i+1): dict(showticklabels = False) for i in range(components.shape[1])})
fig.update_layout({"yaxis"+str(i+1): dict(showticklabels = False, title='PC'+str(i+1)) for i in range(components.shape[1])})


fig.update_layout(margin={"r":150,"t":20,"l":150,"b":20})

fig.show()

/home/ric/Documents/Code/hackathonBBVA/venv/lib/python3.10/site-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.



In [63]:
data_pca = PCA()
data_pca.fit(ordinal_data.drop(out, axis=1))
X = (ordinal_data.drop(out, axis=1).to_numpy()).T
components = (data_pca.components_[:8,:]@X).T

components.shape

(27426, 8)

In [65]:
components_df = pd.DataFrame(components, columns=['PC'+str(i+1)+' ({:.1f})%'.format(data_pca.explained_variance_ratio_[i]*100) for i in range(8)])
#components_df.insert(0, 'Valorcomercial', trainer_data['Valorcomercial'].to_numpy())
#components_df.insert(0, 'Valorcomercial', ordinal_data[raw_data_master['Provincia'] == 'Lima']['Valorcomercial'])

fig = px.scatter_matrix(components_df,
    dimensions=['PC'+str(i+1)+' ({:.1f})%'.format(data_pca.explained_variance_ratio_[i]*100) for i in range(8)],
    color=ordinal_data['Distrito'],#np.array([val if val < 200 else 300 for val in mape_list]),
    title="PCA Pairs plot",
    labels=components_df.columns) # remove underscore
fig.update_traces(diagonal_visible=False)
fig.update_traces(marker_size=1)

fig.update_layout(font=dict(size=10))

fig.update_layout({"xaxis"+str(i+1): dict(showticklabels = False) for i in range(8)})
fig.update_layout({"yaxis"+str(i+1): dict(showticklabels = False, title='PC'+str(i+1)) for i in range(8)})


fig.update_layout(margin={"r":150,"t":20,"l":150,"b":20})

fig.show()

/home/ric/Documents/Code/hackathonBBVA/venv/lib/python3.10/site-packages/plotly/express/_core.py:137: FutureWarning:

Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.

